In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Constant_Maturity_ED.csv',index_col = 'Date')
df.index = pd.to_datetime(df.index)

In [3]:
df.head()

,ED1,ED2,ED3,ED4,ED5,ED6,ED7,ED8,ED9,ED10,ED11,ED12,ED13,ED14,ED15,ED16,ED17,ED18,ED19,ED20
Date,,,,,,,,,,,,,,,,,,,,
2000-01-03,6.300146,6.579726,6.805882,7.018287,7.028155,7.119748,7.163734,7.230232,7.184041,7.195232,7.205515,7.265000,7.235150,7.259495,7.278118,7.358092,7.354708,7.396914,7.427265,7.506268
2000-01-04,6.275171,6.532367,6.763403,6.963487,6.973894,7.065366,7.109766,7.180379,7.133714,7.145372,7.155975,7.210109,7.180000,7.204755,7.223725,7.303605,7.299734,7.342448,7.372636,7.452906
2000-01-05,6.294548,6.582542,6.820957,7.014204,7.030129,7.126494,7.175916,7.250987,7.213686,7.225527,7.241706,7.300245,7.269865,7.295000,7.319284,7.404037,7.399817,7.442871,7.477671,7.564385
2000-01-06,6.292558,6.563636,6.792313,6.974441,6.990855,7.087533,7.142152,7.215723,7.173301,7.185680,7.202299,7.260325,7.229766,7.255227,7.275000,7.359521,7.354867,7.398412,7.433124,7.521096
2000-01-07,6.276060,6.525951,6.753566,6.918970,6.937110,7.032297,7.077964,7.156076,7.112993,7.125816,7.142907,7.200386,7.169677,7.195443,7.215701,7.300000,7.294925,7.338949,7.373583,7.462833


Problem 1

In [4]:
# Training Sample
SampleA = df['2011-01-01':'2015-01-01']

In [5]:
import statsmodels.api as sm

def CCA_Chou_Ng(data_set):
    
    #data_set is pandas dataframe
    df_lag = data_set.shift(1).dropna()
    df = data_set.drop(data_set.index[0]).dropna()
    n = len(data_set.columns)
    
    #X(t) ~ M_1 + X(t-1)
    X = df_lag.as_matrix()
    X_I = sm.add_constant(X) #assure there is a constant term
    Y = df.as_matrix()
    l1 = sm.OLS(Y,X_I).fit()
    B=l1.params[1:(n+1)]
    
    #X(t-1) ~ M_2 + X(t)
    Y_I = sm.add_constant(X)
    l2 = sm.OLS(X,Y_I).fit()
    A=l2.params[1:(n+1)]
    C = np.dot(A,B)
    eig_val, eig_vec = np.linalg.eig(C)
    return eig_val, eig_vec, C # Notice each column rather than row in eig_vec is an eigenvector 

In [6]:
# 5 Pairs 
Pairslist = [['ED8','ED12'],['ED12','ED16'],['ED16','ED20'],['ED8','ED16'],['ED12','ED20']]

# weights list corresponding to pairs
wlist = []
for pair in Pairslist:
    val, vec, C = CCA_Chou_Ng(SampleA[pair])
    w = vec[:,val.argmin()]
    wlist.append(w)
print(wlist)

[array([-0.86614623,  0.49979066]), array([-0.79869269,  0.60173914]), array([-0.70990087,  0.70430161]), array([-0.93135808,  0.36410455]), array([-0.72668716,  0.68696854])]


In [7]:
from statsmodels.tsa.arima_model import ARMA

# Construct Signal 1
# Parameters list
paralist1 = []

for i in range(len(wlist)):
    
    # Signal 1 cointegrated vector
    citg_vec = SampleA[Pairslist[i]].dot(wlist[i])
    
    # AR(1) Model fit
    model = ARMA(citg_vec,(1,0)).fit(ic = 'aic',trend = 'c')
    params = model.params.values
    
    # Calculate half-life
    half_life = -np.log(2)/np.log(params[1])
    
    # Append parameters
    paralist1.append(np.append(params,half_life))

# Print out AR(1) fitting results and halflife
print('Signal 1: ')
pp = ['(2y,3y)','(3y,4y)','(4y,5y)','(2y,4y)','(3y,5y)']
print(pd.DataFrame(paralist1,index=pp,columns=['beta0','beta1','half_life']))

Signal 1: 
            beta0     beta1  half_life
(2y,3y) -0.057478  0.981400  36.917515
(3y,4y)  0.036673  0.985892  48.785330
(4y,5y)  0.393135  0.721335   2.121981
(2y,4y) -0.144839  0.984056  43.127318
(3y,5y)  0.804406  0.929740   9.514734


In [8]:
# Calculate EWMA series whcih admit lambda>1
def EWMA(series,lambda_):
    ewma = series[0]
    ewmalist = [ewma]
    for i in range(1,len(series)):
        ewma = ewma*lambda_+series[i]*(1-lambda_)
        ewmalist.append(ewma)
    return np.array(ewmalist)

In [9]:
# Given lambda, return the distance of halflife and 5
def lambdaerror(lambda_):
    citg_ewm = citg_vec - EWMA(citg_vec,lambda_)
    model = ARMA(citg_ewm,(1,0)).fit(ic = 'aic',trend = 'c')
    params = model.params
    half_life = -np.log(2)/np.log(abs(params[1]))
    return abs(5-half_life)

In [10]:
from scipy.optimize import minimize
# Construct Signal 2
# Parameters list
paralist2 = []

for i in range(len(wlist)):
    
    # Signal 1 cointegrated vector
    citg_vec = (SampleA[Pairslist[i]].values).dot(wlist[i])
    
    # Find lambda which assures halflife of signal2 is 5
    minimizer = minimize(lambdaerror,x0=0.94,method='Nelder-Mead')
    lambda_ =  minimizer['x']
    
    # Signal 2 cointegrated vector = Signal1 - EWMA 
    citg_vec = citg_vec - EWMA(citg_vec,lambda_)
    
    # AR(1) Model fit
    model = ARMA(citg_vec,(1,0)).fit(ic = 'aic',trend = 'c')
    params = model.params
    
    # Calculate halflife which should be 5
    half_life = -np.log(2)/np.log(params[1])
     
    # Append parameters
    paralist2.append(np.append(np.append(params,half_life),lambda_))

# Print out AR(1) fitting results, lambda and halflife
print('Signal 2: ')
pp = ['(2y,3y)','(3y,4y)','(4y,5y)','(2y,4y)','(3y,5y)']
print(pd.DataFrame(paralist2,index=pp,columns=['beta0','beta1','half_life','lambda']))    

C:\Users\xj537\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Signal 2: 
            beta0     beta1  half_life    lambda
(2y,3y) -0.005175  0.870547   4.999872  0.954802
(3y,4y) -0.004915  0.870553   5.000112  0.951541
(4y,5y)  0.167007  0.870547   4.999852  1.004874
(2y,4y) -0.007958  0.870557   5.000249  0.953184
(3y,5y) -0.048562  0.870548   4.999893  0.991520


In [11]:
# Validation Sample
SampleB = df['2015-01-01':'2016-01-01']

In [12]:
def AR1forecast(current, beta0, beta1, H):
    # current can be a vector
    forecast = current
    for i in range(H):
        forecast = forecast*beta1 + beta0
    return forecast

In [13]:
from scipy.stats import pearsonr

def metric(signal, forecast):
    
    # Remove useless data in start and end. Remove data where difference is less than 0.1
    signal_slice = signal[10:]
    forecast_slice = forecast[:-10]
    signal_comp = signal_slice[abs(signal_slice-forecast_slice)>0.1]
    forecast_comp = forecast_slice[abs(signal_slice-forecast_slice)>0.1]    
    
    # calculate residuals
    residuals = signal_comp - forecast_comp
    
    # calculate pearson correlation
    corr = pearsonr(signal_comp,forecast_comp)[0]
    
    # calculate root mean squared error
    rmse = np.sqrt((residuals*2).mean())
    
    return corr, rmse

In [14]:
def QualityAnalysis(w, Sample):

    corrlist = []
    rmselist = []
    
    for i in range(len(Pairslist)):

        # Biuld Signal1, Signal2 and Signal3. Signal3 is gaussian mixture of Signal1&2 with weights theta(w,1-w).
        signal1 = (Sample[Pairslist[i]].values).dot(wlist[i])
        signal2 = signal1 - EWMA(signal1,paralist2[i][3])
        signal3 = w*signal1 + (1-w) *signal2     

        # Forecast horizon H, which means forecasting H days after
        H = 10

        # Forecast of signal1 where horizon is H
        forecast1 = AR1forecast(signal1, paralist1[i][0], paralist1[i][1], H)

        # Forecast of signal2 where horizon is H
        forecast2 = AR1forecast(signal2, paralist2[i][0], paralist2[i][1], H)

        # Forecast of signal3 where horizon is H and weights is (w,1-w)
        forecast3 = w*forecast1 + (1-w)*forecast2

        # calculate metrics
        corr1, ms1 = metric(signal1, forecast1)
        corr2, ms2 = metric(signal2, forecast2)
        corr3, ms3 = metric(signal3, forecast3)
        corrlist.append([corr1, corr2, corr3])
        rmselist.append([ms1, ms2, ms3])

    # calculate mean of all cointegrated pairs
    ave_corr = pd.DataFrame(corrlist).mean().values
    ave_rmse = pd.DataFrame(rmselist).mean().values
    
    metrics = pd.DataFrame(np.array([ave_corr,ave_rmse]).T, index=['signal1','signal2','signal3'],columns=['CORR','RMSE'])
    
    return metrics

In [15]:
QualityAnalysis(0.5, SampleB)

,CORR,RMSE
signal1,0.706694,1.272676
signal2,0.684891,0.551710
signal3,0.604775,0.920395


In [16]:
def toOptimizew(w, Sample):
    
    # Rescale metrics
    metrics = QualityAnalysis(w, Sample)
    corr_rescale = metrics.iloc[:,1]/metrics.iloc[:,1].mean()
    RMSE_rescale = metrics.iloc[:,0]/metrics.iloc[:,0].mean()
    
    # Combine two metrics to optimize signal3 weight. The smaller combined metirc, the better model is.
    CombMetric = sum(abs(RMSE_rescale/corr_rescale))
    #CombMetric = sum(abs(metrics.iloc[:,1]/metrics.iloc[:,0]))
    
    return CombMetric

In [17]:
minimizer = minimize(toOptimizew,0.6,args=(SampleB),bounds=[(0,1)])
Optimalw = minimizer['x']
print('Optimal W: ', Optimalw)

Optimal W:  [0.35543532]


In [18]:
# Test Sample
SampleC = df['2016-01-01':'2017-01-01']

In [19]:
# Devide into four quarters
SampleC1 = SampleC['2016-01-01':'2016-04-01']
SampleC2 = SampleC['2016-04-01':'2016-07-01']
SampleC3 = SampleC['2016-07-01':'2016-10-01']
SampleC4 = SampleC['2016-10-01':'2017-01-01']

In [20]:
# Quality analysis on test sample
print('Q1:')
print(QualityAnalysis(Optimalw, SampleC1))
print('Combined Metric: ', toOptimizew(Optimalw, SampleC1))
print('\n')

print('Q2:')
print(QualityAnalysis(Optimalw, SampleC2))
print('Combined Metric: ', toOptimizew(Optimalw, SampleC2))
print('\n')

print('Q3:')
print(QualityAnalysis(Optimalw, SampleC3))
print('Combined Metric: ', toOptimizew(Optimalw, SampleC3))
print('\n')

print('Q4:')
print(QualityAnalysis(Optimalw, SampleC4))
print('Combined Metric: ', toOptimizew(Optimalw, SampleC4))
print('\n')

Q1:
             CORR      RMSE
signal1  0.215136  1.303990
signal2  0.345577  0.595858
signal3  0.205381  0.849959
Combined Metric:  3.541233653231919


Q2:
             CORR      RMSE
signal1  0.111295  1.287085
signal2  0.514837  0.563672
signal3  0.140456  0.799511
Combined Metric:  4.064009434578916


Q3:
             CORR      RMSE
signal1  0.213459  1.267783
signal2 -0.066878  0.680515
signal3 -0.200113  0.761108
Combined Metric:  26.80296196096009


Q4:
             CORR      RMSE
signal1  0.534266  1.223124
signal2  0.332559  0.777767
signal3  0.454130  0.723741
Combined Metric:  3.0771485495619926




Comments:
    We should expect that as time passes, our signal forecast quality should decrease. But in my results, we can see quality increases in the first 3 season. It may reflect our model is not that robust. But we can see quality does decrease in the 4th quarter, it may imply our model is fine. The first 3 season may be some accidently things that our model doesn't learn. Maybe our model will be more robust if we use more traing datas.

Problem 2

In [21]:
# 2F Short Rate Model:
# r(t) = x1(t) + x2(t)
# dxi(t) = (ui - kixi(t))dt + sigmai dWit,  i = 1,2
# dW1t dW2t = rou dt

class Model:
    def __init__(self,x1,x2,mu1,mu2,k1,k2,sigma1,sigma2,rou):
        self.mu1 =mu1
        self.mu2 = mu2
        self.k1 = k1
        self.k2 = k2
        self.sigma1 = sigma1
        self.sigma2 = sigma2
        self.rou = rou
        self.x1 = x1
        self.x2 = x2
        
    # Conditional mean of X1(t) given X1(0)=x1   
    def mean_x1(self,t): 
        m = self.x1*np.exp(-self.k1*t) + self.mu1/self.k1*(1-np.exp(-self.k1*t))
        return m
    
    # Conditional mean of X2(t) given X2(0)=x1
    def mean_x2(self,t):
        m = self.x2*np.exp(-self.k2*t) + self.mu2/self.k2*(1-np.exp(-self.k2*t))
        return m
    
    # Conditional var of X1(t) given X1(0)=x1
    def var_x1(self,t):
        v = self.sigma1**2/2/self.k1*(1-np.exp(-2*self.k1*t))
        return v
    
    # Conditional var of X2(t) given X2(0)=x1
    def var_x2(self,t):
        v = self.sigma2**2/2/self.k2*(1-np.exp(-2*self.k2*t))
        return v
    
    def A(self,tau):
        c1 = self.mu1/self.k1+self.mu2/self.k2-self.sigma1**2/2/self.k1**2-self.sigma2**2/2/self.k2**2-self.sigma1*self.sigma2*self.rou/self.k1/self.k2
        c2 = self.sigma1**2/self.k1**3+self.sigma1*self.sigma2*self.rou/2/self.k1**2/self.k2-self.mu1/self.k1**2
        c3 = self.sigma2**2/self.k2**3+self.sigma1*self.sigma2*self.rou/2/self.k2**2/self.k1-self.mu2/self.k2**2
        c4 = self.sigma1**2/4/self.k1**3
        c5 = self.sigma2**2/4/self.k2**3
        c6 = self.sigma1*self.sigma2*self.rou/self.k1/self.k2/(self.k1+self.k2)
        a1 = c1*tau+c2*(1-np.exp(-self.k1*tau))+c3*(1-np.exp(-self.k2*tau))
        a2 = c4*(1-np.exp(-2*self.k1*tau)) + c5*(1-np.exp(-2*self.k2*tau)) +c6*(1-np.exp(-(self.k1+self.k2)*tau))
        a = a1 - a2
        return a

    def B1(self,tau):
        b1 = (1-np.exp(-self.k1*tau))/self.k1
        return b1
    
    def B2(self,tau):
        b2 = (1-np.exp(-self.k2*tau))/self.k2
        return b2
    
    # forward rate if forward rate fix time is t_fix, forward rate start time is T0 and forward rate end time is T1
    def fwd_rate(self,t_fix,T0,T1):
        A_diff = self.A(T0-t_fix) - self.A(T1-t_fix)
        B1_diff = self.B1(T0-t_fix) - self.B1(T1-t_fix)
        B2_diff = self.B2(T0-t_fix) - self.B2(T1-t_fix)
        b1 = - B1_diff*self.mean_x1(t_fix) + (B1_diff**2)*self.var_x1(t_fix)/2
        b2 = - B2_diff*self.mean_x2(t_fix) + (B2_diff**2)*self.var_x2(t_fix)/2
        bcross = B1_diff*B2_diff*self.rou*np.sqrt(self.var_x1(t_fix)*self.var_x2(t_fix))
        f = (np.exp(-A_diff + b1 + b2 + bcross) - 1)/(T1-T0)
        return f
    
    # Update latent variable
    def update_x(self,x):
        self.x1 = x[0]
        self.x2 = x[1]
        
    # Update measure relavent variable
    def update_p(self,p):
        self.mu1 = p[0]
        self.mu2 = p[1]
        self.k1 = p[2]
        self.k2 = p[3]
    
    # Update measure irrelavent variable
    def update_c(self,c):
        self.sigma1 = c[0]
        self.sigma2 = c[1]
        self.rou = c[2]

In [22]:
from scipy.optimize import least_squares, leastsq
class xFitter:
    
    # This class is a inner loop to fit latent variable x to real data fut_data if other parameters are given
    def __init__(self,model,fut_data,terms):
        self.model = model
        self.data = np.array(fut_data)
        self.terms = terms
    
    # residuals of fitting
    def residuals(self,x):
        m = self.model
        m.update_x(x)
        rate_dist = [m.fwd_rate(t,t,t+0.25)*100 for t in self.terms]
        return np.array(rate_dist) - self.data
    
    # perform fitting using least square method
    def fit(self,x0,solver='lm'):
        return least_squares(self.residuals,x0,method=solver,xtol = 0.1)

In [23]:
class pFitter:
    
    # This class is a outer loop to fit p to real data fut_data while fiting x is inner loop and c is updated by x 
    def __init__(self,model,sample,terms):
        self.model = model
        self.sample = sample
        self.terms = terms
    
    # Fit x for all days and return x series and residuals dataframe
    def Fitallx(self,p):
        
        m = self.model
        m.update_p(p)
        
        # Store optimal x for each day into a list
        xlist=[]

        # Store residuals for each day and each futurerate into a list
        residuallist = []
        
        for ix in self.sample.index:
            
            # fit x for each day
            xfitter = xFitter(m,self.sample.loc[ix],self.terms)
            optimalx = xfitter.fit([0.02,0.02])['x']
            xlist.append(optimalx)
            residuallist.append(xfitter.residuals(optimalx))
        
        xseries = pd.DataFrame(xlist,index=self.sample.index,columns=['x1','x2'])
        residuals = pd.DataFrame(residuallist,index=self.sample.index,columns=self.sample.columns)
        return xseries,residuals
    
    # Calculate Jacobian Matrix
    def Jacobian(self,p,initial):
        
        Jacobian = []
        for i in range(len(p)):
            
            # For each parameter, calculate derivative
            p[i] = p[i] + 0.000001
            SSE = ((self.Fitallx(p)[1])**2).mean().mean()
            J = (SSE - initial)/0.000001
            Jacobian.append(J)
            p[i] = p[i] - 0.000001
        
        return np.array(Jacobian)
        
    # perform fit
    def fit(self, p0, solver='lm'):
        
        p = p0
        SSE = 0
        preSSE = 1000
        
        # Control terminal
        while abs(SSE-preSSE)>0.01:
            
            # See parameters in each loop
            preSSE = SSE
            #print(SSE)
            #print('p: ',p)
            #print('c: ',[self.model.sigma1,self.model.sigma2,self.model.rou])
            
            # fit x assume p and c given
            xseries, residuals = self.Fitallx(p)
            
            # Calculate sum squared error
            SSE = (residuals**2).mean().mean()
            
            # Calculate Jacobian and Hessian
            Jac = self.Jacobian(p,SSE)
            Hess = Jac.reshape(-1,1).dot(Jac.reshape(1,-1))
           
            # Detect collinearity, if condition number is greater than 1000
            eig = np.linalg.eig(Hess)[0]
            condnum = abs(eig.max()/eig.min())
            
            # Use ridge method to handle collinearity 
            if condnum>1000:
                Hess = Hess  + 0.00001*np.eye(len(Jac))
            
            # Calculate c using simulated x series
            dx = xseries.diff().dropna()
            sigma1 = dx.iloc[:,0].std()
            sigma2 = dx.iloc[:,1].std()
            corr =dx.corr().iloc[0,1]
            
            # Update parameters
            self.model.update_c([sigma1,sigma2,corr])
            p = p - np.linalg.inv(Hess).dot(Jac)*SSE
            
        return p, [self.model.sigma1,self.model.sigma2,self.model.rou], residuals       

In [24]:
x1 = 0.01
x2 = 0.02
mu1 = 0.01
mu2 = 0.008
k1 = 0.08
k2 = 0.12
sigma1 = 0.02
sigma2 = 0.02
rou = 0.5
model = Model(x1,x2,mu1,mu2,k1,k2,sigma1,sigma2,rou)
term = [xx/4. for xx in range(1,21)]
pfitter = pFitter(model,SampleA,term)
fitresult = pfitter.fit(np.array([0.01,0.008,0.08,0.12]))

In [25]:
print('Optimal parameters p: ', fitresult[0])
print('Optimal parameters c: ', fitresult[1])

Optimal parameters p:  [0.01434384 0.01092208 0.07919113 0.11847654]
Optimal parameters c:  [0.008537503457144009, 0.008668464150088241, -0.9988814238967613]


In [26]:
# Calculate half life for a given series
def half_life(data_srs):
    # data_set is pandas Series
    # Assume data_set follows an O-U process: dX = lambda*(X_bar-X)dt+sigma*dW
    data_srs_demean = data_srs - data_srs.mean()
    df_lag = data_srs_demean.shift(1).dropna()
    df = data_srs_demean.drop(data_srs.index[0]).dropna()
    lambdaa = -np.log(np.dot(df,df_lag)/np.dot(df,df)) # estimate lambda in O-U process
    hl = np.log(2)/lambdaa
    return hl # Here hl is in scale of days

In [28]:
from statsmodels.tsa.stattools import adfuller as adf
from scipy.stats import kstest

# property list
proplist = []

# Residuals of fitting
residuals = fitresult[2]

# analyze properties for residuals
for col in residuals.columns:
    series = residuals[col]
    
    # Stationary property
    adftest = adf(series,autolag='t-stat') 
    
    # Half life
    hl = half_life(series)
    
    # volitality
    vol = series.std()
    
    # if residuals are normal distributed
    isnorm = kstest(series,'norm')
    
    # append properties
    proplist.append([adftest[1],hl,vol,isnorm[1]])

properties = ['ADF P Value','Half Life','Volitality','Norm Test P Value']    
propTest = pd.DataFrame(proplist,index = residuals.columns,columns = properties)
print(propTest)


      ADF P Value  Half Life  Volitality  Norm Test P Value
ED1      0.979475  33.769997    0.177851                0.0
ED2      0.965145  37.755365    0.149216                0.0
ED3      0.828680  40.529724    0.117496                0.0
ED4      0.499365  30.421056    0.086066                0.0
ED5      0.649988  38.816911    0.069492                0.0
ED6      0.860959  28.182600    0.090343                0.0
ED7      0.869071  41.285425    0.123780                0.0
ED8      0.930435  31.572359    0.147726                0.0
ED9      0.919446  31.613033    0.160656                0.0
ED10     0.911592  29.645691    0.162587                0.0
ED11     0.869613  27.752027    0.152979                0.0
ED12     0.848191  26.841753    0.132033                0.0
ED13     0.747907  14.467384    0.103712                0.0
ED14     0.538571  56.060850    0.070846                0.0
ED15     0.377579  15.395334    0.043804                0.0
ED16     0.568509   3.164632    0.051684

Some comments here:
    This result shows that our 2-Factor Vasicek Model is not a good fit. The residuals are neither stationary nor normally 
distributed. Perhaps if our loop goes futher we can have a better fit. But my computer dont have that calculation ability so far.

In [29]:
# Repeat Problem 1

# Construct Signal 1
# Parameters list
paralist1 = []

for i in range(len(wlist)):
    
    # Signal 1 cointegrated vector
    citg_vec = residuals[Pairslist[i]].dot(wlist[i])
    
    # AR(1) Model fit
    model = ARMA(citg_vec,(1,0)).fit(ic = 'aic',trend = 'c')
    params = model.params.values
    
    # Calculate half-life
    half_life = -np.log(2)/np.log(params[1])
    
    # Append parameters
    paralist1.append(np.append(params,half_life))

# Print out AR(1) fitting results and halflife
print('Signal 1: ')
pp = ['(2y,3y)','(3y,4y)','(4y,5y)','(2y,4y)','(3y,5y)']
print(pd.DataFrame(paralist1,index=pp,columns=['beta0','beta1','half_life']))

from scipy.optimize import minimize
# Construct Signal 2
# Parameters list
paralist2 = []

for i in range(len(wlist)):
    
    # Signal 1 cointegrated vector
    citg_vec = (SampleA[Pairslist[i]].values).dot(wlist[i])
    
    # Find lambda which assures halflife of signal2 is 5
    minimizer = minimize(lambdaerror,x0=0.94,method='Nelder-Mead')
    lambda_ =  minimizer['x']
    
    # Signal 2 cointegrated vector = Signal1 - EWMA 
    citg_vec = citg_vec - EWMA(citg_vec,lambda_)
    
    # AR(1) Model fit
    model = ARMA(citg_vec,(1,0)).fit(ic = 'aic',trend = 'c')
    params = model.params
    
    # Calculate halflife which should be 5
    half_life = -np.log(2)/np.log(params[1])
     
    # Append parameters
    paralist2.append(np.append(np.append(params,half_life),lambda_))

# Print out AR(1) fitting results, lambda and halflife
print('Signal 2: ')
pp = ['(2y,3y)','(3y,4y)','(4y,5y)','(2y,4y)','(3y,5y)']
print(pd.DataFrame(paralist2,index=pp,columns=['beta0','beta1','half_life','lambda']))    

print(QualityAnalysis(0.5, SampleB))

minimizer = minimize(toOptimizew,0.6,args=(SampleB),bounds=[(0,1)])
Optimalw = minimizer['x']
print('Optimal W: ', Optimalw)

# Quality analysis on test sample
print('Q1:')
print(QualityAnalysis(Optimalw, SampleC1))
print('Combined Metric: ', toOptimizew(Optimalw, SampleC1))
print('\n')

print('Q2:')
print(QualityAnalysis(Optimalw, SampleC2))
print('Combined Metric: ', toOptimizew(Optimalw, SampleC2))
print('\n')

print('Q3:')
print(QualityAnalysis(Optimalw, SampleC3))
print('Combined Metric: ', toOptimizew(Optimalw, SampleC3))
print('\n')

print('Q4:')
print(QualityAnalysis(Optimalw, SampleC4))
print('Combined Metric: ', toOptimizew(Optimalw, SampleC4))
print('\n')

Signal 1: 
            beta0     beta1  half_life
(2y,3y) -0.094720  0.987473  54.982760
(3y,4y) -0.057513  0.986417  50.683737
(4y,5y) -0.013771  0.799965   3.105675
(2y,4y) -0.135541  0.988137  58.084089
(3y,5y) -0.086914  0.937223  10.691093


C:\Users\xj537\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Signal 2: 
            beta0     beta1  half_life    lambda
(2y,3y) -0.005175  0.870547   4.999872  0.954802
(3y,4y) -0.004915  0.870553   5.000112  0.951541
(4y,5y)  0.167007  0.870547   4.999852  1.004874
(2y,4y) -0.007958  0.870557   5.000249  0.953184
(3y,5y) -0.048562  0.870548   4.999893  0.991520
             CORR      RMSE
signal1  0.706694  1.161632
signal2  0.684891  0.551710
signal3  0.602909  0.966183
Optimal W:  [0.23731254]
Q1:
             CORR      RMSE
signal1  0.215136  1.173165
signal2  0.345577  0.595858
signal3  0.157800  0.778967
Combined Metric:  3.4253531637645818


Q2:
             CORR      RMSE
signal1  0.111295  1.153562
signal2  0.514837  0.563672
signal3  0.078910  0.724750
Combined Metric:  3.874806343568943


Q3:
             CORR      RMSE
signal1  0.213459  1.130475
signal2 -0.066878  0.680515
signal3 -0.261903  0.688266
Combined Metric:  14.468693615571539


Q4:
             CORR      RMSE
signal1  0.534266  1.116553
signal2  0.332559  0.777767
signal